In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

# Homework
Note: sometimes your answer doesn't match one of the options exactly. That's fine. Select the option that's closest to your solution.

In this homework, we will use the Car price dataset like last week. Download it from here.

Or you can do it with wget:

wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
We'll work with the MSRP variable, and we'll transform it to a classification task.

For the rest of the homework, you'll need to use only these columns:

Make,
Model,
Year,
Engine HP,
Engine Cylinders,
Transmission Type,
Vehicle Style,
highway MPG,
city mpg
MSRP

In [ ]:
df_full = pd.read_csv("https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv")

In [ ]:
df_full.head()

In [ ]:
columns_to_keep = ['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders', 'Transmission Type', 'Vehicle Style', 'highway MPG', 'city mpg', 'MSRP']

df = df_full[columns_to_keep]
df.head()

# Data preparation
Keep only the columns above
Lowercase the column names and replace spaces with underscores
Fill the missing values with 0
Make the price binary (1 if above the average, 0 otherwise) - this will be our target variable above_average
Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution. Use train_test_split function for that with random_state=1

In [ ]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
#df

In [ ]:
df.rename(columns = {'msrp':'price'}, inplace = True) 
#df

In [ ]:
# Which columns/features have NAs/Nulls?
df.isnull().sum()

In [ ]:
# Replace NAs by zero
df.fillna(0, inplace=True)
df.isnull().sum()

# Question 1: ROC AUC feature importance
ROC AUC could also be used to evaluate feature importance of numerical variables.

Let's do that

For each numerical variable, use it as score and compute AUC with the above_average variable
Use the training dataset for that
If your AUC is < 0.5, invert this variable by putting "-" in front

(e.g. -df_train['engine_hp'])

AUC can go below 0.5 if the variable is negatively correlated with the target varialble. You can change the direction of the correlation by negating this variable - then negative correlation becomes positive.

Which numerical variable (among the following 4) has the highest AUC?

engine_hp
engine_cylinders
highway_mpg
city_mpg

In [ ]:

# Let's create a variable above_average which is 1 if the price is above its mean value and 0 otherwise.
df['above_average'] = (df.price > df.price.mean()).astype(int)
#df['above_average']

In [ ]:
df_for_training = df.copy()
del df_for_training['price']
categorical = ['make', 'model', 'transmission_type', 'vehicle_style']
numerical = ['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'price']

df_train_full, df_test = train_test_split(df_for_training, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=1)
y_train = df_train.above_average.values
y_val = df_val.above_average.values
y_test = df_test.above_average.values

del df_train['above_average']
del df_val['above_average']
del df_test['above_average']

In [ ]:
df_train

In [ ]:
numerical = numerical[:-1] # Drop 'price' from features 

# CALCULATING ROC AUC SCORE PER FEATURE:
for c in numerical:
    c_above_average = (df_train[c]>=df_train[c].mean()).astype(int)

    auc = roc_auc_score(y_train, c_above_average)
    if auc < 0.5:
        auc = roc_auc_score(y_train, -c_above_average)
        
    print('%s, %.3f' % (c, auc))

<b> engine_hp has the highest AUC </b>

Some additional analysis to verify that engine_hp has the highest AUC

In [ ]:
from sklearn.metrics import roc_auc_score

model_one_feature = LogisticRegression(solver='liblinear', C=1, max_iter=1000, random_state=1)
model_one_feature.fit(np.array(df_train['engine_hp']).reshape(-1, 1),y_train)
y_pred = model_one_feature.predict(np.array(df_val['engine_hp']).reshape(-1, 1))
engine_hp_accuracy = round(accuracy_score(y_val, y_pred),3)
print(f"Accuracy on the validation dataset with just engine_hp: {engine_hp_accuracy}")

engine_hp_auc = round(roc_auc_score(y_val, y_pred),3)
print(f"AUC on the validation dataset with just engine_hp: {engine_hp_auc}")

In [ ]:
model_one_feature = LogisticRegression(solver='liblinear', C=1, max_iter=1000, random_state=1)
model_one_feature.fit(np.array(df_train['engine_cylinders']).reshape(-1, 1),y_train)
y_pred = model_one_feature.predict(np.array(df_val['engine_cylinders']).reshape(-1, 1))
engine_cylinders_accuracy = round(accuracy_score(y_val, y_pred),3)
print(f"Accuracy on the validation dataset with engine_cylinders: {engine_cylinders_accuracy}")

engine_cylinders_auc = round(roc_auc_score(y_val, y_pred),3)
print(f"AUC on the validation dataset with just engine_cylinders: {engine_cylinders_auc}")

In [ ]:
model_one_feature = LogisticRegression(solver='liblinear', C=1, max_iter=1000, random_state=1)
model_one_feature.fit(np.array(df_train['highway_mpg']).reshape(-1, 1),y_train)
y_pred = model_one_feature.predict(np.array(df_val['highway_mpg']).reshape(-1, 1))
highway_mpg_accuracy = round(accuracy_score(y_val, y_pred),3)
print(f"Accuracy on the validation dataset with just highway_mpg: {highway_mpg_accuracy}")

highway_mpg_auc = round(roc_auc_score(y_val, y_pred),3)
print(f"AUC on the validation dataset with just highway_mpg: {highway_mpg_auc}")

In [ ]:
model_one_feature = LogisticRegression(solver='liblinear', C=1, max_iter=1000, random_state=1)
model_one_feature.fit(np.array(df_train['city_mpg']).reshape(-1, 1),y_train)
y_pred = model_one_feature.predict(np.array(df_val['city_mpg']).reshape(-1, 1))
city_mpg_accuracy = round(accuracy_score(y_val, y_pred),3)
print(f"Accuracy on the validation dataset with just city_mpg: {city_mpg_accuracy}")

city_mpg_auc = round(roc_auc_score(y_val, y_pred),3)
print(f"AUC on the validation dataset with just highway_mpg: {city_mpg_auc}")

# Question 2: Training the model
Apply one-hot-encoding using DictVectorizer and train the logistic regression with these parameters:

LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
What's the AUC of this model on the validation dataset? (round to 3 digits)

0.678
0.779
0.878
0.979

In [ ]:
columns = categorical + numerical

train_dicts = df_train[columns].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

val_dicts = df_val[columns].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_pred = model.predict_proba(X_val)[:, 1]
     
auc_val = round(roc_auc_score(y_val, y_pred),3)     
print(f"AUC on the validation dataset: {auc_val}")

In [ ]:
fpr, tpr, thresholds = roc_curve(y_val, y_pred)
print(f"AUC on the validation dataset: {auc_val}")


plt.figure(figsize=(5, 5))

plt.plot(fpr, tpr, color='black')
plt.plot([0, 1], [0, 1], color='black', lw=0.7, linestyle='dashed', alpha=0.5)

plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.title('ROC curve')

plt.show()

<b> AUC on the validation dataset is 0.979 </b>

# Question 3: Precision and Recall
Now let's compute precision and recall for our model.

Evaluate the model on all thresholds from 0.0 to 1.0 with step 0.01
For each threshold, compute precision and recall
Plot them
At which threshold precision and recall curves intersect?

0.28
0.48
0.68
0.88

In [ ]:
y_pred = model.predict_proba(X_val)[:, 1]

scores = []

thresholds = np.linspace(0, 1, 101)

for t in thresholds: #B
    tp = ((y_pred >= t) & (y_val == 1)).sum()
    fp = ((y_pred >= t) & (y_val == 0)).sum()
    fn = ((y_pred < t) & (y_val == 1)).sum()
    tn = ((y_pred < t) & (y_val == 0)).sum()
    scores.append((t, tp, fp, fn, tn))

df_scores = pd.DataFrame(scores)
df_scores.columns = ['threshold', 'tp', 'fp', 'fn', 'tn']

In [ ]:
df_scores

In [ ]:
df_scores['tpr'] = df_scores.tp / (df_scores.tp + df_scores.fn)
df_scores['fpr'] = df_scores.fp / (df_scores.fp + df_scores.tn)

In [ ]:
plt.figure(figsize=(6, 4))

plt.plot(df_scores.threshold, df_scores.tpr, color='black', linestyle='solid', label='TPR')
plt.plot(df_scores.threshold, df_scores.fpr, color='black', linestyle='dashed', label='FPR')
plt.legend()

plt.xticks(np.linspace(0, 1, 11))
plt.yticks(np.linspace(0, 1, 11))

plt.xlabel('Thresholds')
plt.title('TPR and FPR')

plt.show()

In [ ]:
df_scores

In [ ]:
df_scores['precision'] = df_scores.tp/(df_scores.tp + df_scores.fp)
df_scores['recall'] = df_scores.tp/(df_scores.tp + df_scores.fn)
df_scores

In [ ]:
plt.figure(figsize=(6, 4))

plt.plot(df_scores.threshold, df_scores.precision, color='black', linestyle='solid', label='Precision')
plt.plot(df_scores.threshold, df_scores.recall, color='black', linestyle='dashed', label='Recall')
plt.legend()

plt.xticks(np.linspace(0, 1, 11))
plt.yticks(np.linspace(0, 1, 11))

plt.xlabel('Thresholds')
plt.title('Precision and Recall')

plt.show()

In [ ]:
df_scores.iloc[47:51]

<b> The precision and recall curves intersect at threshold = 0.48 (nearest answer) </b>

# Question 4: F1 score
Precision and recall are conflicting - when one grows, the other goes down. That's why they are often combined into the F1 score - a metrics that takes into account both

This is the formula for computing F1:

 

Where 
 is precision and 
 is recall.

Let's compute F1 for all thresholds from 0.0 to 1.0 with increment 0.01

At which threshold F1 is maximal?

0.12
0.32
0.52
0.72

In [ ]:
df_scores['F1'] = (2* df_scores.precision * df_scores.recall)/(df_scores.precision + df_scores.recall)

In [ ]:
df_scores

In [ ]:
df_scores.iloc[np.linspace(12, 72, 7)]

In [ ]:
max(df_scores.F1)

max_F1_idx = df_scores["F1"].idxmax()
optimal_threshold = df_scores.loc[max_F1_idx, "threshold"]

print(f"The maximal F1 score is {df_scores['F1'][max_F1_idx]:.2f} at threshold {optimal_threshold:.2f}")

In [ ]:
df_scores.iloc[40:59]

<b> F1 is maximal at threshold = 0.52 (nearest answer) </b>



Question 5: 5-Fold CV
--------------------------
Use the KFold class from Scikit-Learn to evaluate our model on 5 different folds:

KFold(n_splits=5, shuffle=True, random_state=1)
Iterate over different folds of df_full_train
Split the data into train and validation
Train the model on train with these parameters: LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
Use AUC to evaluate the model on validation
How large is standard devidation of the scores across different folds?

0.003
0.030
0.090
0.140

In [ ]:
def train(df, y):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)

    X = dv.transform(cat)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
    model.fit(X, y)

    return dv, model


def predict(df, dv, model):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(cat)

    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

aucs = []

for train_idx, val_idx in kfold.split(df_train_full):
    df_train = df_train_full.iloc[train_idx]
    y_train = df_train.above_average

    df_val = df_train_full.iloc[val_idx]
    y_val = df_val.above_average

    dv, model = train(df_train, y_train)
    y_pred = predict(df_val, dv, model)

    rocauc = roc_auc_score(y_val, y_pred)
    aucs.append(rocauc)
    
np.array(aucs).round(3)

print('auc = %0.3f ± %0.3f' % (np.mean(aucs), np.std(aucs)))


<b> The standard devidation of the scores across different folds is 0.003 (nearest answer) </b>

# Question 6: Hyperparemeter Tuning

Now let's use 5-Fold cross-validation to find the best parameter C

Iterate over the following C values: [0.01, 0.1, 0.5, 10]
Initialize KFold with the same parameters as previously
Use these parametes for the model: LogisticRegression(solver='liblinear', C=C, max_iter=1000)
Compute the mean score as well as the std (round the mean and std to 3 decimal digits)
Which C leads to the best mean score?

0.01
0.1
0.5
10
If you have ties, select the score with the lowest std. If you still have ties, select the smallest C.

In [ ]:
def train(df, y, C=1.0):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)

    X = dv.transform(cat)

    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000)
    model.fit(X, y)

    return dv, model

def predict(df, dv, model):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(cat)

    y_pred = model.predict_proba(X)[:, 1]

    return y_pred


nfolds = 5
kfold = KFold(n_splits=nfolds, shuffle=True, random_state=1)



for C in [0.01,0.1,0.5,10]:
    aucs = []

    for train_idx, val_idx in kfold.split(df_train_full):
        df_train = df_train_full.iloc[train_idx]
        y_train = df_train.above_average
        
        df_val = df_train_full.iloc[val_idx]
        y_val = df_val.above_average
        
        dv, model = train(df_train, y_train, C)
        y_pred = predict(df_val, dv, model)
        
        auc = roc_auc_score(y_val, y_pred)
        aucs.append(auc)

    print('C=%s, auc = %0.3f ± %0.3f' % (C, np.mean(aucs), np.std(aucs)))

<b> C = 10 gives the highest AUC </b>